<a href="https://colab.research.google.com/github/riccardo247/CodeRL/blob/actor_model/Code_Lint_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**switch runtime to GPU**

**LOAD all the libraries and code**



1.   codeRL
2.   transformers
1.   model
2.   toenizer




In [18]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 20 22:08:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    32W /  70W |   4237MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
!git clone --branch src_layout https://github.com/SteveJu/AA228_final_project.git

Cloning into 'AA228_final_project'...
remote: Enumerating objects: 12025, done.
remote: Counting objects: 100% (2175/2175), done.
remote: Compressing objects: 100% (1552/1552), done.
remote: Total 12025 (delta 569), reused 2168 (delta 564), pack-reused 9850
Receiving objects: 100% (12025/12025), 24.13 MiB | 23.33 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [20]:
!pip install -r /content/AA228_final_project/CodeRL_copy/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import os
os.getcwd()
os.listdir()

['images',
 'README.md',
 'CODEOWNERS',
 'CODE_OF_CONDUCT.md',
 'AA228_final_project',
 'SECURITY.md',
 'trainers',
 'extract_example_test.ipynb',
 'models',
 'datasets',
 'generate.py',
 'transformers',
 'test_one_solution.py',
 'requirements.txt',
 'utils',
 'data',
 'configs',
 'LICENSE.txt',
 'train.py',
 'scripts',
 'outputs']

In [22]:
##download finetuned code from google bucket
%cd /content/AA228_final_project/CodeRL_copy
!mkdir models
%cd ./models
!mkdir codet5_finetuned_codeRL
%cd codet5_finetuned_codeRL
!gsutil cp gs://bigbird-freefly/coderl/codet5-large-rlfinetuned/config.json ./
!gsutil cp gs://bigbird-freefly/coderl/codet5-large-rlfinetuned/pytorch_model.bin ./
!ls

/content/AA228_final_project/CodeRL_copy
mkdir: cannot create directory ‘models’: File exists
/content/AA228_final_project/CodeRL_copy/models
mkdir: cannot create directory ‘codet5_finetuned_codeRL’: File exists
/content/AA228_final_project/CodeRL_copy/models/codet5_finetuned_codeRL
Copying gs://bigbird-freefly/coderl/codet5-large-rlfinetuned/config.json...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      
Copying gs://bigbird-freefly/coderl/codet5-large-rlfinetuned/pytorch_model.bin...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][  2.8 GiB/  2.8 GiB]   82.4 MiB/s                                   
Operation completed over 1 objects/2.8 GiB.                          

In [23]:
#get the tokenizer
%cd /content/AA228_final_project/CodeRL_copy/
!mkdir ./models/codet5_tokenizer
%cd ./models/codet5_tokenizer
!wget https://huggingface.co/Salesforce/codet5-large/blob/main/tokenizer.json
!wget https://huggingface.co/Salesforce/codet5-large/blob/main/tokenizer_config.json
!wget https://huggingface.co/Salesforce/codet5-large/blob/main/special_tokens_map.json
!wget https://huggingface.co/Salesforce/codet5-large/blob/main/vocab.json

/content/AA228_final_project/CodeRL_copy
mkdir: cannot create directory ‘./models/codet5_tokenizer’: File exists
/content/AA228_final_project/CodeRL_copy/models/codet5_tokenizer
--2023-03-20 22:09:42--  https://huggingface.co/Salesforce/codet5-large/blob/main/tokenizer.json
Resolving huggingface.co (huggingface.co)... 23.20.207.15, 52.2.178.255, 35.173.225.216, ...
Connecting to huggingface.co (huggingface.co)|23.20.207.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54941 (54K) [text/html]
Saving to: ‘tokenizer.json.2’

tokenizer.json.2    100%[===================>]  53.65K  --.-KB/s    in 0.08s   

2023-03-20 22:09:43 (632 KB/s) - ‘tokenizer.json.2’ saved [54941/54941]

--2023-03-20 22:09:43--  https://huggingface.co/Salesforce/codet5-large/blob/main/tokenizer_config.json
Resolving huggingface.co (huggingface.co)... 35.173.225.216, 34.203.133.210, 23.20.50.97, ...
Connecting to huggingface.co (huggingface.co)|35.173.225.216|:443... connected.
HTTP request

**DEMO:**
select one input and run all the steps:
 to  

1.   initialize one of the input text and tokenize
2.   generate code
1.    detokenize code





 , 


In [24]:
text_input = "QUESTION:\
              Nicholas has an array a that contains n distinct integers from 1 to n. In other words, Nicholas has a permutation of size n.\
              Nicholas want the minimum element (integer 1) and the maximum element (integer n) to be as far as possible from each other. \
              He wants to perform exactly one swap in order to maximize the distance between the minimum and the maximum elements. \
              The distance between two elements is considered to be equal to the absolute difference between their positions.\
              ANSWER:\
              "


In [11]:
text_input = "QUESTION:\
              Apart from having lots of holidays throughout the year, residents of Berland also have whole lucky years. \
              Year is considered lucky if it has no more than 1 non-zero digit in its number. So years 100, 40000, 5 are lucky and 12, 3001 and 12345 are not.\
              You are given current year in Berland. Your task is to find how long will residents of Berland wait till the next lucky year.\
              Use Standard Input format\
              ANSWER:\
              "

In [25]:
#install ocal transformers
%cd /content/AA228_final_project/CodeRL_copy/transformers
%pip install -e .
%cd /content/AA228_final_project/CodeRL_copy/

from transformers import RobertaTokenizer, T5ForConditionalGeneration
import torch 
#prepare inputs tokenize
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base', cache_dir='./models/codet5_tokenizer')
model = model = T5ForConditionalGeneration.from_pretrained('./models/codet5_finetuned_codeRL/', local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



/content/AA228_final_project/CodeRL_copy/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/AA228_final_project/CodeRL_copy/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.1
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers
/content/AA228_final_project/CodeRL_copy


T5ForConditionalGeneration(
  (shared): Embedding(32100, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

In [26]:
#run the model to generate the code
input_ids = torch.LongTensor(tokenizer.encode(text_input, 
                                              verbose=False, 
                                              max_length=512)).unsqueeze(0).cuda()
output_ids = model.generate(
                        input_ids, 
                        do_sample=True, 
                        temperature=0.3, 
                        max_length=512, 
                        num_return_sequences=1,
                        top_p=0.95)   

In [27]:
#decode and print the code generated
code_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print('python code generated:')
print(code_output)

python code generated:
n=2**(n+1)
	x=x[0]
	y=x[1]
	for i in x:
		if(i==0):
			continue
		if(i==n)
			continue
		x[i]=x[i-1]+1
		else:
			y[i]=y[i-1]+1
		x[i]=y[i-1]+1
	return x

